In [4]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [5]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

train.head(5)

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


### BMI Feature 생성

In [6]:
train['BMI'] = train['Weight'] / (train['Height']**2)
test['BMI'] = test['Weight'] / (test['Height']**2)

### 명목형 변수 인코딩

In [7]:
# CAEC 변수 인코딩 수정
caec_mapping = {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
train['CAEC'] = train['CAEC'].map(caec_mapping)
test['CAEC'] = test['CAEC'].map(caec_mapping)

# CALC 변수 인코딩
calc_mapping = {'no': 0, 'Sometimes': 1, 'Frequently': 2}
train['CALC'] = train['CALC'].map(calc_mapping)
test['CALC'] = test['CALC'].map(calc_mapping)

# MTRANS 변수 인코딩
mtrans_mapping = {
    'Walking': 4,  # 가장 활동량이 많음
    'Bike': 3,  # 상당한 활동량
    'Public_Transportation': 2,  # 일정한 활동량
    'Motorbike': 1,  # 적은 활동량
    'Automobile': 0  # 가장 적은 활동량
}
train['MTRANS'] = train['MTRANS'].map(mtrans_mapping)
test['MTRANS'] = test['MTRANS'].map(mtrans_mapping)

# NObeyesdad 변수 인코딩
nobeyesdad_mapping = {
    'Insufficient_Weight': 0,
    'Normal_Weight': 1,
    'Overweight_Level_I': 2,
    'Overweight_Level_II': 3,
    'Obesity_Type_I': 4,
    'Obesity_Type_II': 5,
    'Obesity_Type_III': 6
}
train['NObeyesdad'] = train['NObeyesdad'].map(nobeyesdad_mapping)

In [8]:
categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']

train_encoded = pd.get_dummies(train, columns=categorical_features)
test_encoded = pd.get_dummies(test, columns=categorical_features)

train_encoded.columns

Index(['id', 'Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CAEC', 'CH2O', 'FAF',
       'TUE', 'CALC', 'MTRANS', 'NObeyesdad', 'BMI', 'Gender_Female',
       'Gender_Male', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes', 'SMOKE_no',
       'SMOKE_yes', 'SCC_no', 'SCC_yes'],
      dtype='object')

## 모델 학습

In [9]:
X_train = train_encoded.drop(['NObeyesdad'], axis=1)
y_train = train_encoded['NObeyesdad']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [10]:
# 모델 초기화
model = XGBClassifier()

# 모델 학습
model.fit(X_train, y_train)

# 검증 데이터로 예측
y_pred = model.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print("검증 데이터 정확도:", accuracy)

검증 데이터 정확도: 0.904383429672447
